# 0.5 - Self Implementation
# 0.5 - SkLearn package

In [115]:
import numpy as np
import pandas as pd

import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch import nn

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import torch.utils.data as Data
from sklearn import datasets
import random 
import os
from keras.layers import Dense, Convolution2D, UpSampling2D, MaxPooling2D, ZeroPadding2D, Flatten, Dropout, Reshape
from keras.models import Sequential
from keras.utils import np_utils


In [116]:
TempData=pd.read_csv('../data/Cancer/dataR2.csv')
TempData.head()
TempData.columns

Index([u'Age', u'BMI', u'Glucose', u'Insulin', u'HOMA', u'Leptin',
       u'Adiponectin', u'Resistin', u'MCP.1', u'Classification'],
      dtype='object')

In [117]:
d=np.asarray(TempData.values)
np.random.shuffle(d)
print d[1]
head="Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification"
np.savetxt("cancerData.csv", d, delimiter=",",header=head)

[ 65.        32.05      97.         5.73       1.370998  61.48
  22.54      10.33     314.05       2.      ]


In [126]:
data=pd.read_csv('./cancerData.csv')
data.head(1)
dataVal=data.values
type(data.loc[0][:9].values)
labels=data['Classification']
data=data.drop({'Classification'},axis=1)
data.head()

,# Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1
0,69,28.444444,108,8.808,2.346451,14.7485,5.288025,16.48508,353.568
1,65,32.050000,97,5.730,1.370998,61.4800,22.540000,10.33000,314.050
2,34,31.975015,87,4.530,0.972138,28.7502,7.642760,5.62592,572.783
3,64,22.222222,98,5.700,1.377880,12.1905,4.783985,13.91245,395.976
4,52,30.801249,87,30.212,6.483495,29.2739,6.268540,24.24591,764.667


In [119]:
# leng=np.array(data.loc[115])
# #print type(leng)
# trains=[np.asarray(['Age' 'BMI' 'Glucose' 'Insulin'  'HOMA'  'Leptin'  
#        'Adiponectin'  'Resistin'  'MCP.1'  'Classification'])]
# for i in range(115):
#     trains.append(np.asarray(data.loc[i]))

# #print trains
# np.savetxt("cancer.csv", trains, delimiter=",")
# cancer=pd.read_csv('./cancer.csv')
# cancer.head()


In [154]:
K=5

In [155]:
#function which cal the distance btw 2 points
def eucladiean_distance(v1,v2):
    return np.sqrt(np.sum((v1-v2)**2))

#loop to calc the distances from each point
def KNNtemp(train,labels,test_x,k=K):
    dist=[]
    
    for ix in range(len(train)): #loop runs for each row in train
        d=eucladiean_distance(train[ix][:9],test_x)
        #we add the distance as well as the lable to tell that this point belongs to which cluster
        dist.append([d,ix,labels[ix]])
    sorted_distances = sorted(dist,key=lambda x:x[0]) #returns a list
    candidates = np.asarray(sorted_distances)[:k,-1] #the labes of the strting k rows are extracted
    
    freq=np.unique(candidates,return_counts=True) 
    #print freq[0]
    index=np.argmax(freq[1])
    print freq
    return freq,index,sorted_distances

def knn(train,labels,test_x):
    freq,index,distances=KNNtemp(train,labels,test_x)
    return freq[0][index]

In [156]:
ToTest=np.asarray([ 48. ,23.5 ,70.,2.707,0.46740867,8.8071,9.7024,7.99585,417.114])
#ToTest=[ 75.,30.48,152.,7.01,2.62828267,50.53,0.06,11.73,99.45]
split = int(0.8 * data.shape[0])
# s=int(0.2 * data.shape[0])
training_data = data[:split]
training_label=labels[:split]
testing_data = data[split:]
testing_label=labels[split:]
print split
print training_data.shape
print type(testing_data)


92
(92, 9)
<class 'pandas.core.frame.DataFrame'>


In [157]:
# print knn(trains[:][:9],trains[:1],ToTest)
print eucladiean_distance(ToTest,training_data.loc[0][:9].values)
print '*'*20
freq,index,sorted_distances = KNNtemp(training_data.values,training_label.values,ToTest,K)
# print freq
# print index
# print sorted_distances

78.19941569630372
********************
(array([1., 2.]), array([1, 4]))


In [158]:
print knn(training_data.values,training_label.values,ToTest)

(array([1., 2.]), array([1, 4]))
2.0


In [159]:
(labels[1])

2.0

In [160]:
loc=0
accuracy=0
for i in testing_data.values:
    pred=knn(training_data.values,training_label.values,i)
    actual=testing_label.values[loc]
    print pred," : ",actual
    if pred==actual:
        accuracy+=1
    loc+=1

(array([1., 2.]), array([3, 2]))
1.0  :  2.0
(array([1., 2.]), array([2, 3]))
2.0  :  2.0
(array([1., 2.]), array([3, 2]))
1.0  :  2.0
(array([2.]), array([5]))
2.0  :  1.0
(array([1., 2.]), array([3, 2]))
1.0  :  2.0
(array([1., 2.]), array([2, 3]))
2.0  :  1.0
(array([1., 2.]), array([2, 3]))
2.0  :  2.0
(array([1., 2.]), array([2, 3]))
2.0  :  1.0
(array([1., 2.]), array([2, 3]))
2.0  :  2.0
(array([1., 2.]), array([1, 4]))
2.0  :  2.0
(array([1., 2.]), array([3, 2]))
1.0  :  2.0
(array([1., 2.]), array([2, 3]))
2.0  :  2.0
(array([1., 2.]), array([3, 2]))
1.0  :  1.0
(array([1., 2.]), array([1, 4]))
2.0  :  1.0
(array([1., 2.]), array([1, 4]))
2.0  :  2.0
(array([1., 2.]), array([3, 2]))
1.0  :  1.0
(array([1., 2.]), array([4, 1]))
1.0  :  1.0
(array([1., 2.]), array([2, 3]))
2.0  :  2.0
(array([1., 2.]), array([1, 4]))
2.0  :  1.0
(array([1., 2.]), array([1, 4]))
2.0  :  1.0
(array([1., 2.]), array([2, 3]))
2.0  :  2.0
(array([1., 2.]), array([3, 2]))
1.0  :  2.0
(array([1., 2.]),

In [161]:
print accuracy/float(testing_data.shape[0])

0.5


In [162]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [163]:
from sklearn.neighbors import KNeighborsClassifier

In [175]:
KNN=KNeighborsClassifier()
KNN.fit(training_data,training_label)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [176]:
KNN.score(testing_data,testing_label)

0.5